# Genetic Algorithm

In [17]:
import numpy as np
from itertools import permutations,islice
import random
import pandas as pd
import copy

In [18]:
class City:
  def __init__(self,city,x,y):
    self.city = city
    self.x = x
    self.y = y
  def __repr__(self) -> str:
      return f'(id:{self.city})'

  def __eq__(self, other):
      return (self.city == other.city)

In [19]:
class Chromosome:
  def __init__(self,c_list,fitness=0, cost=0):
    self.c_list = c_list
    self.fitness = fitness
    self.cost = cost
  def __repr__(self) -> str:
      return f'({self.c_list}, {self.cost})'

In [20]:
data_df = pd.read_csv('15-Points.csv')

In [21]:
def calculate_distance(point1,point2):
    return np.sqrt((point1.x-point2.x)**2+(point1.y-point2.y)**2)

In [22]:
def generate_distance_matrix(chromosome):
    distance_matrix = np.zeros((len(chromosome.c_list),len(chromosome.c_list)))
    for i in range(len(chromosome.c_list)):
        for j in range(i,len(chromosome.c_list)):
            dist = calculate_distance(chromosome.c_list[i],chromosome.c_list[j])
            distance_matrix[i][j] = distance_matrix[j][i] = dist

        distance_matrix[i][i] = np.inf
    distance_matrix_dict = dict(zip([i.city for i in chromosome.c_list], distance_matrix.tolist()))

    return copy.deepcopy(distance_matrix_dict)
# generate_distance_matrix(generate_intial_population(3,data_df)[0])

In [23]:
def calculate_cost(c_list,distance_matrix_dict):
  cost=0
  for i in range(1,len(c_list)):
      cost += distance_matrix_dict[c_list[i-1].city][c_list[i].city-1]
  cost += distance_matrix_dict[c_list[0].city][c_list[-1].city-1]
  return cost

In [24]:
def calculate_fitness(cost):
    return 1/cost

In [25]:
def generate_intial_population(popuation_size,intial_chromosome,distance_matrix_dict):
  pop = []
  for i in range(popuation_size):
      c_list = np.random.permutation(intial_chromosome.c_list)
      cost = calculate_cost(c_list,distance_matrix_dict)
      pop.append(Chromosome(c_list,fitness=calculate_fitness(cost),cost=cost))

  return copy.deepcopy(np.array(pop))
# generate_intial_population(3,data_df)[0].c_list[0].city

In [26]:
def elitism(old_pop,elit_size):
  
    sorted_pop = copy.deepcopy(np.array(sorted(old_pop, key=lambda chrom: chrom.cost)))

    return sorted_pop[:elit_size]

In [27]:
def selection(pop,k):
  parent_pool = np.random.choice(pop,size=k,replace=False)
  return copy.deepcopy(sorted(parent_pool, key=lambda chrom: chrom.fitness, reverse=True)[:2])

In [28]:
def crossover_process(parent1, parent2, slice_point1, slice_point2):
    child1 = copy.deepcopy(parent1)
    child2 = copy.deepcopy(parent2)
    
    for i in range(slice_point1, slice_point2):
        temp1 = child1.c_list[i]
        
        indx = np.where(child2.c_list==temp1)[0][0]
        
        child2.c_list[i], child2.c_list[indx] = copy.deepcopy(child2.c_list[indx]), copy.deepcopy(child2.c_list[i])

    return copy.deepcopy(child2)

In [29]:
def crossover(pop,slice_point1,slice_point2,new_pop_size,distance_matrix_dict):
    # number of k tournements
    k=5
    new_pop = []
    for i in range(new_pop_size):
        parent_1, parent_2 = selection(pop,k)
        child_1 = crossover_process(parent_1, parent_2, slice_point1, slice_point2)

        child_1.cost = calculate_cost(child_1.c_list,distance_matrix_dict)
        child_1.fitness = calculate_fitness(child_1.cost)
        
        new_pop.append(child_1)
    return copy.deepcopy(new_pop)

In [30]:
def mutation(new_pop,mutation_rate,distance_matrix_dict):

    for indx in np.random.choice(list(range(len(new_pop))),size=int(len(new_pop)*mutation_rate),replace=False):

        i,j = np.random.choice(list(range(len(new_pop[indx].c_list))),size=2)
        new_pop[indx].c_list[i], new_pop[indx].c_list[j] = new_pop[indx].c_list[j], new_pop[indx].c_list[i]

        new_pop[indx].cost = calculate_cost(new_pop[indx].c_list,distance_matrix_dict)
        #review
        new_pop[indx].fitness = calculate_fitness(new_pop[indx].cost)

    return copy.deepcopy(new_pop)

In [31]:
def GA_process(data_df,popuation_size,elit_size,mutation_rate,iterations):
    intial_chromosome = Chromosome(c_list=[City(int(row[1][2]),row[1][0],row[1][1]) for row in data_df.iterrows()]) 
    distance_matrix_dict = generate_distance_matrix(intial_chromosome)
    pop = generate_intial_population(popuation_size,intial_chromosome,distance_matrix_dict)
#review
    slice_point1,slice_point2 = 4, 10

    for i in range(iterations):

        elitism_pop = elitism(pop,elit_size)

        new_pop_size = popuation_size - elit_size
        new_pop = crossover(pop,slice_point1,slice_point2,new_pop_size,distance_matrix_dict)

        new_pop = mutation(copy.deepcopy(new_pop),mutation_rate,distance_matrix_dict)

        pop = np.array(copy.deepcopy(elitism_pop.tolist()) + copy.deepcopy(new_pop))

    return list(sorted(pop, key=lambda chrom: chrom.fitness, reverse=True))[0].cost

In [33]:
GA_process(data_df,popuation_size=200,elit_size=20,mutation_rate=0.7,iterations=40)

284.3810904080331